In [21]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [22]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain L3H,Snow L3H
0,0,Manoharpur,IN,22.35,85.21,68.76,85,4,4.12,clear sky,0.0,0.0
1,1,Rikitea,PF,-23.12,-134.97,73.15,82,95,16.28,overcast clouds,0.0,0.0
2,2,Tura,RU,64.27,100.22,-17.79,91,100,2.86,overcast clouds,0.0,0.0
3,3,Punta Arenas,CL,-53.16,-70.91,50.00,57,95,10.29,overcast clouds,0.0,0.0
4,4,Richards Bay,ZA,-28.77,32.06,69.66,94,100,23.62,heavy intensity rain,20.5,0.0


In [23]:
# Ask the customer to add a minimum and maximum temperature value, snow and rain preferences.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
want_rain = input("Do you want it to be raining? (yes/no) ")
want_snow = input("Do you want it to be snowing? (yes/no) ")


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [24]:
temp_condition = (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)

if want_rain == "yes":    
    rain_condition = city_data_df["Rain L3H"] > 0
else:
    rain_condition = city_data_df["Rain L3H"] == 0


if want_snow == "yes":    
    snow_condition = city_data_df["Snow L3H"] > 0
else:
    snow_condition = city_data_df["Snow L3H"] == 0

    
preferred_cities_df = city_data_df.loc[rain_condition & snow_condition & temp_condition] 

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain L3H,Snow L3H
7,7,Avdon,IL,33.05,35.18,80.60,47,20,6.93,few clouds,0.0,0.0
13,13,Arraial Do Cabo,BR,-22.97,-42.02,82.40,74,75,8.05,broken clouds,0.0,0.0
14,14,Touros,BR,-5.20,-35.46,86.00,55,40,14.99,scattered clouds,0.0,0.0
15,15,Atuona,PF,-9.80,-139.03,79.32,70,3,12.97,clear sky,0.0,0.0
26,26,Kapaa,US,22.08,-159.32,75.20,73,90,4.70,overcast clouds,0.0,0.0


In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng","Weather Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
7,Avdon,IL,80.60,33.05,35.18,few clouds,
13,Arraial Do Cabo,BR,82.40,-22.97,-42.02,broken clouds,
14,Touros,BR,86.00,-5.20,-35.46,scattered clouds,
15,Atuona,PF,79.32,-9.80,-139.03,clear sky,
26,Kapaa,US,75.20,22.08,-159.32,overcast clouds,
33,Faanui,PF,79.93,-16.48,-151.75,overcast clouds,
38,Port-Gentil,GA,80.60,-0.72,8.78,scattered clouds,
39,Tocopilla,CL,77.11,-22.09,-70.20,overcast clouds,
40,Ponta Do Sol,BR,85.91,-20.63,-46.00,overcast clouds,
47,Dwarka,IN,75.34,28.58,77.04,broken clouds,


In [26]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
7,Avdon,IL,80.60,33.05,35.18,few clouds,Travel Hotel Eilon
13,Arraial Do Cabo,BR,82.40,-22.97,-42.02,broken clouds,Pousada Porto Praia
14,Touros,BR,86.00,-5.20,-35.46,scattered clouds,INN NEW HORIZON
15,Atuona,PF,79.32,-9.80,-139.03,clear sky,Villa Enata
26,Kapaa,US,75.20,22.08,-159.32,overcast clouds,Courtyard by Marriott Kaua'i at Coconut Beach
33,Faanui,PF,79.93,-16.48,-151.75,overcast clouds,Bora-Bora Pearl Beach Resort & Spa
38,Port-Gentil,GA,80.60,-0.72,8.78,scattered clouds,Fengshui Residence
39,Tocopilla,CL,77.11,-22.09,-70.20,overcast clouds,Hotel Galvarino
40,Ponta Do Sol,BR,85.91,-20.63,-46.00,overcast clouds,Hotel Escarpas do Lago
47,Dwarka,IN,75.34,28.58,77.04,broken clouds,Radisson Blu New Delhi Dwarka


In [27]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description/dt><dd>{Weather Description}</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")